In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 读取图像并检查
image_path = "111.png"
image = cv2.imread(image_path)
if image is None:
    raise ValueError(f"Image at path '{image_path}' not found or cannot be loaded.")

# 复制图像用于显示红点标记
image_with_dots = image.copy()

# 转换为灰度图
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 高斯模糊以减少噪声
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# 使用 Canny 边缘检测
low_threshold = 20
high_threshold = 100
edges = cv2.Canny(blurred, low_threshold, high_threshold)

# 形态学操作增强边缘
kernel = np.ones((3, 3), np.uint8)
edges = cv2.dilate(edges, kernel, iterations=1)
edges = cv2.erode(edges, kernel, iterations=1)

# 找到轮廓
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 初始化顶点坐标
corners = []

# 遍历轮廓，寻找具有四个顶点的轮廓
for contour in contours:
    # 使用多边形逼近轮廓
    epsilon = 0.02 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    
    # 检查是否为四边形
    if len(approx) == 4:
        corners = approx
        break

# 如果找到四个角点，输出坐标
if len(corners) == 4:
    # 将角点排序为左上、右上、右下和左下
    corners = sorted(corners, key=lambda x: (x[0][1], x[0][0]))
    top_points = sorted(corners[:2], key=lambda x: x[0][0])   # 左上、右上
    bottom_points = sorted(corners[2:], key=lambda x: x[0][0]) # 左下、右下
    sorted_corners = np.array([top_points[0], top_points[1], bottom_points[1], bottom_points[0]])
    
    # 输出角点坐标，并在 image_with_dots 上标注红点
    for i, point in enumerate(sorted_corners):
        x, y = point[0]
        print(f"Corner {i + 1}: (x={x}, y={y})")
        cv2.circle(image_with_dots, (x, y), 10, (0, 0, 255), -1)  # 使用红色标注

    # 定义裁剪后的图像尺寸
    width = int(np.linalg.norm(sorted_corners[0][0] - sorted_corners[1][0]))
    height = int(np.linalg.norm(sorted_corners[0][0] - sorted_corners[3][0]))

    # 定义目标点，顺序与 sorted_corners 对应
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype="float32")

    # 计算透视变换矩阵
    M = cv2.getPerspectiveTransform(sorted_corners.astype("float32"), dst_points)

    # 进行透视变换以裁剪图像
    cropped_image = cv2.warpPerspective(image, M, (width, height))

    # 保存裁剪后的图像，不包含红点
    cv2.imwrite("temmap.png", cropped_image)

    # 显示带红点的图像
    plt.imshow(cv2.cvtColor(image_with_dots, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title("Image with Marked Corners")
    plt.show()

    # 显示裁剪后的图像
    plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title("Cropped Image")
    plt.show()
else:
    print("未检测到四个角点，无法裁剪。")


未检测到四个角点，无法裁剪。


In [7]:
import cv2
import numpy as np

# 读取图像
image_path = "222.png"
image = cv2.imread(image_path)
if image is None:
    raise ValueError(f"Image at path '{image_path}' not found or cannot be loaded.")

# 转换为灰度并进行预处理
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(cv2.GaussianBlur(gray, (5, 5), 0), 20, 100)
edges = cv2.erode(cv2.dilate(edges, np.ones((3, 3), np.uint8), iterations=1), None, iterations=1)

# 寻找四边形轮廓
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
corners = next((cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)
                for contour in contours if len(cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)) == 4), None)

# 如果找到四个角点，按顺序排列并裁剪图像
if corners is not None:
    corners = sorted(corners, key=lambda x: (x[0][1], x[0][0]))
    top_points = sorted(corners[:2], key=lambda x: x[0][0])
    bottom_points = sorted(corners[2:], key=lambda x: x[0][0])
    sorted_corners = np.array([top_points[0], top_points[1], bottom_points[1], bottom_points[0]], dtype="float32")

    width, height = int(np.linalg.norm(sorted_corners[0][0] - sorted_corners[1][0])), int(np.linalg.norm(sorted_corners[0][0] - sorted_corners[3][0]))
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype="float32")

    # 透视变换裁剪图像
    cropped_image = cv2.warpPerspective(image, cv2.getPerspectiveTransform(sorted_corners, dst_points), (width, height))

    # 去除四周各 6 个像素并保存
    cropped_image_borderless = cropped_image[5:-5, 5:-5]
    cv2.imwrite("temmap.png", cropped_image)
    cv2.imwrite("crop_map.png", cropped_image_borderless)
else:
    print("未检测到四个角点，无法裁剪。")
